In [1]:
from gamestate import GameState, start_ai_only_game
from models.RandomModel import RandomModel
from models.HeuristicModel import HeuristicModel
from models.DeepRLModel import DeepRLModel
from trainutil import train_models, test_models
import fsutils as fs

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
env = GameState()
deep_rl_model = DeepRLModel()
# rand_model_1 = RandomModel(min_steps=5, max_steps=10)
# rand_model_2 = RandomModel(min_steps=5, max_steps=10)
models = [deep_rl_model]

print(deep_rl_model.model)

DQN(
  (fc1): Linear(in_features=46, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=2048, bias=True)
  (fc3): Linear(in_features=2048, out_features=8, bias=True)
  (relu): ReLU()
)


In [4]:
train_models(env, models, episodes=20, steps=2000, print_every=2000)
# test_models(env, models, steps=2000)


TRAIN MODE
----STEP 0 rewards----
Model 0: -0.020000000000010232
------EPISODE 0 rewards------
Model 0: -20.535551903640766
----STEP 0 rewards----
Model 0: 0.9799999999999898
------EPISODE 1 rewards------
Model 0: -28.040186312574633
----STEP 0 rewards----
Model 0: 0.9799999999999898
----LEARNING BEGINS----
------EPISODE 2 rewards------
Model 0: -20.256779838322103
----STEP 0 rewards----
Model 0: -0.020000000000010232
[1703] [VIRUS] Agent3 ate virus 22
------EPISODE 3 rewards------
Model 0: 84.11238964678986
----STEP 0 rewards----
Model 0: 1.9799999999999898
------EPISODE 4 rewards------
Model 0: -16.77428144842611
----STEP 0 rewards----
Model 0: 0.9799999999999898
[263] [VIRUS] Agent5 ate virus 31
------EPISODE 5 rewards------
Model 0: 100.96995777196057
----STEP 0 rewards----
Model 0: 1.9799999999999898
[1485] [VIRUS] Agent6 ate virus 38
------EPISODE 6 rewards------
Model 0: 134.8004529754162
----STEP 0 rewards----
Model 0: 2.9799999999999898
[327] [VIRUS] Agent7 ate virus 43
-----

In [5]:
fs.save_net_to_disk(deep_rl_model.model, "test_drl_10_step3000")